In [1]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree
import re

### Código para archivo XML

In [2]:
#Abre archivo xml que contiene informacion de las alineaciones
with open('condoy_aligned.xml', 'rt') as f:
    tree = ElementTree.parse(f)

#Crea lista para almacenar los xtargets
xtarget = [] 
#Extrae el atributo xtargets de link
for node in tree.findall('.//link'):
    xtarget.append(node.attrib.get('xtargets'))
    
#Busca elemetos con doble target
for i in range(len(xtarget)):
    for j in xtarget[i]:
        if j == " ":
            xtarget[i] = " ; "
                    
#Divide el contenido de la lista usando como delimitador ;
def divide(lista):
    for i in range(len(lista)):
        lista[i]=lista[i].split(";")
    return lista

#Divide xtargets es y mir
xtarget = divide(xtarget)

es = []
mir = []
for i in range(len(xtarget)):
    es.append(xtarget[i][0])
for i in range(len(xtarget)):
    mir.append(xtarget[i][1])
    
es = pd.DataFrame(es,columns=['xtargets_es'])
es['xtargets_es'].replace(' ', np.nan, inplace=True)
es['xtargets_es'].replace('', np.nan, inplace=True)

mir = pd.DataFrame(mir,columns=['xtargets_mir'])
mir['xtargets_mir'].replace(' ', np.nan, inplace=True)
mir['xtargets_mir'].replace('', np.nan, inplace=True)

#Concatena es y mir
es_mir = pd.concat([es,mir], axis = 1)
es_mir = es_mir.dropna()

#Convierte xtargets a enteros
es_mir = es_mir.astype('int64')
es_mir = es_mir.reset_index(drop=True)

es = es_mir['xtargets_es']
es = pd.DataFrame(es,columns=['xtargets_es'])
mir = es_mir['xtargets_mir']
mir = pd.DataFrame(mir,columns=['xtargets_mir'])

#### Código para convertir a un dataframe los dos archivos que fueron alineados para juntarlo
#### con sus xtargets correspondientes con la función merge y generar un txt con las alineaciones

In [3]:
#Archivo que contiene frases en español
frases_es = pd.read_csv("condoy_es.txt", header=None, sep=".")
frases_es.dropna(subset=[0], inplace=True)

ID = []
for i in range(1,len(frases_es)+1):
    ID.append(i)

id_ = pd.DataFrame(ID,columns=['id_es'])
id_frases = pd.concat([id_,frases_es], axis = 1)

#Hace un filtro para descartar las frases que no tienen alineación otra frase y los que tienen doble alineación
#uniendo los xtargets del español generado por yasa con el identificador de linea de cada frase
df_es = pd.merge(es, id_frases, left_on='xtargets_es', right_on='id_es')


#Archivo que contiene frases en Ayuuk
frases_mir = pd.read_csv("condoy_mir.txt", header=None, sep=".")
frases_mir.dropna(subset=[0], inplace=True)

ID = []
for i in range(1,len(frases_mir)+1):
    ID.append(i)

id_ = pd.DataFrame(ID,columns=['id_mir'])
id_frases = pd.concat([id_,frases_mir], axis = 1)
df_mir = pd.merge(mir, id_frases, left_on='xtargets_mir', right_on='id_mir')

#Extrae columnas que contiene las frase que tuvieron correspondencia
col_es = df_es[[0]]
col_mir = df_mir[[0]]

es_mir_aligned = pd.concat([col_es,col_mir], axis = 1)
es_mir_aligned.columns = ["es","mir"]

es_mir_aligned.to_csv('esmir-alineado.txt', header=False, index=False, sep = ".")